# Imaging inverse problems with adversarial networks

This example shows you how to train various networks using adversarial training for deblurring problems. We demonstrate running training and inference using DeblurGAN, AmbientGAN and UAIR implemented in the `deepinv` library, and how to simply train your own GAN by using `deepinv.training.AdversarialTrainer`. These examples can also be easily extended to train more complicated GANs such as CycleGAN.

- Kupyn et al., [_DeblurGAN: Blind Motion Deblurring Using Conditional Adversarial Networks_](https://openaccess.thecvf.com/content_cvpr_2018/papers/Kupyn_DeblurGAN_Blind_Motion_CVPR_2018_paper.pdf)
- Bora et al., [_AmbientGAN: Generative models from lossy measurements_](https://openreview.net/forum?id=Hy7fDog0b)
- Pajot et al., [_Unsupervised Adversarial Image Reconstruction_](https://openreview.net/forum?id=BJg4Z3RqF7)

Adversarial networks are characterised by the addition of an adversarial loss $\mathcal{L}_\text{adv}$ to the standard reconstruction loss:

$$\mathcal{L}_\text{adv}(x,\hat x;D)=\mathbb{E}_{x\sim p_x}\left[q(D(x))\right]+\mathbb{E}_{\hat x\sim p_{\hat x}}\left[q(1-D(\hat x))\right]$$

where $D(\cdot)$ is the discriminator model, $x$ is the reference image, $\hat x$ is the estimated reconstruction, $q(\cdot)$ is a quality function (e.g $q(x)=x$ for WGAN). Training alternates between generator $f$ and discriminator $D$ in a minimax game. When there are no ground truths (i.e unsupervised), this may be defined on the measurements $y$ instead.

**DeblurGAN** forward pass: 

$$\hat x = f(y)$$

**DeblurGAN** loss: 

$$\mathcal{L}=\mathcal{L}_\text{sup}(\hat x, x)+\mathcal{L}_\text{adv}(\hat x, x;D)$$

where $\mathcal{L}_\text{sup}$ is a supervised loss such as pixel-wise MSE or VGG Perceptual Loss.

**AmbientGAN** forward pass: 

$$\hat x = f(z),\quad z\sim \mathcal{N}(\mathbf{0},\mathbf{I}_k)$$

**AmbientGAN** loss (where $A(\cdot)$ is the physics): 

$$\mathcal{L}=\mathcal{L}_\text{adv}(A(\hat x), y;D)$$

Forward pass at eval time:

$$\hat x = f(\hat z)\quad\text{s.t.}\quad\hat z=\operatorname*{argmin}_z D(A(f(z)),y)$$

**UAIR** forward pass:

$$\hat x = f(y)$$

**UAIR** loss: 

$$\mathcal{L}=\mathcal{L}_\text{adv}(\hat y, y;D)+\lVert A(f(\hat y))- \hat y\rVert^2_2,\quad\hat y=A(\hat x)$$

In [7]:
import deepinv as dinv
from deepinv.loss import adversarial
import torch
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, ToTensor, CenterCrop
from torchvision.datasets.utils import download_and_extract_archive

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

Load data and apply some forward degradation to the images. For this example we use the Urban100 dataset resized to 128x128. For simplicity we apply an isotropic Gaussian blur for demonstration, although the original papers deal with harder inverse problems.

In [2]:
physics = dinv.physics.Blur(dinv.physics.blur.gaussian_blur(sigma=(5, 5)))

In [3]:
download_and_extract_archive(
            "https://huggingface.co/datasets/eugenesiow/Urban100/resolve/main/data/Urban100_HR.tar.gz?download=true",
            "Urban100",
            filename="Urban100_HR.tar.gz",
            md5="65d9d84a34b72c6f7ca1e26a12df1e4c"
        )

100%|██████████| 135388067/135388067 [00:10<00:00, 12587619.01it/s]


Extracting Urban100\Urban100_HR.tar.gz to Urban100


In [8]:
train_dataset, test_dataset = random_split(ImageFolder("Urban100", transform=Compose([ToTensor(), CenterCrop(128)])), (0.8, 0.2))

dataset_path = dinv.datasets.generate_dataset(
    train_dataset= train_dataset,
    test_dataset = test_dataset,
    physics=physics,
    device=device,
    save_dir=f"Urban100",
    )
    
train_dataloader = DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=True), 
    shuffle=True
)
test_dataloader = DataLoader(
    dinv.datasets.HDF5Dataset(dataset_path, train=False),
    shuffle=False
)

Computing train measurement vectors from base dataset...


100%|██████████| 2/2 [00:04<00:00,  2.02s/it]


Computing test measurement vectors from base dataset...


100%|██████████| 5/5 [00:00<00:00,  9.87it/s]

Dataset has been saved in Urban100


Define reconstruction network (i.e. conditional generator) and discriminator network to use for adversarial training. For demonstration we use a simple U-Net as the reconstruction network and the discriminator from [PatchGAN](https://arxiv.org/abs/1611.07004), but these can be replaced with any architecture e.g. transformers, unrolled etc. Further discriminator models are in `deepinv.models.gan`.

In [ ]:
def get_models(lr_g=1e-4, lr_d=1e-4):
    model = dinv.models.UNet(
        in_channels=3, 
        out_channels=3,
        scales=2,
        circular_padding=True,
        batch_norm=False
        )

    D = dinv.models.PatchGANDiscriminator(
        n_layers=2,
        batch_norm=False
    )

    #TODO make sure zero_grad_only right way round for training
    optimizer = dinv.training.adversarial_training.AdversarialOptimizer(
        torch.optim.Adam(model.parameters(), lr=lr_g, weight_decay=1e-8),
        torch.optim.Adam(D.parameters(),     lr=lr_d, weight_decay=1e-8),
    )
    scheduler = dinv.training.adversarial_training.AdversarialScheduler(
        torch.optim.lr_scheduler.StepLR(optimizer.G, step_size=5, gamma=0.9),
        torch.optim.lr_scheduler.StepLR(optimizer.D, step_size=5, gamma=0.9)
    )
    
    return model, D, optimizer, scheduler

### DeblurGAN training

In [ ]:
model, D, optimizer, scheduler = get_models()

Construct pixel-wise and adversarial losses as defined above. We use the MSE for the supervised pixel-wise metric for simplicity but this can be easily replaced with a perceptual loss if desired.

In [ ]:
loss_g = [
    dinv.loss.SupLoss(metric=torch.nn.MSELoss),
    adversarial.DeblurGANGeneratorLoss()
]
loss_d = adversarial.DeblurGANDiscriminatorLoss()

Train the networks using `AdversarialTrainer`

In [ ]:
dinv.training.adversarial_training.AdversarialTrainer(
    D=D,
    epochs=5,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    verbose=True
).train(
    model=model,
    physics=physics,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader
)

### UAIR training

In [ ]:
model, D, optimizer, scheduler = get_models(lr_g=1e-4, lr_d=4e-4) # learning rates from original paper

Construct losses as defined above

In [ ]:
loss_g = adversarial.UAIRGeneratorLoss()
loss_d = adversarial.UAIRDiscriminatorLoss()

Train the networks using `AdversarialTrainer`

In [ ]:
dinv.training.adversarial_training.AdversarialTrainer(
    D=D,
    epochs=5,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    verbose=True
).train(
    model=model,
    physics=physics,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader
)

### AmbientGAN training

In [ ]:
model, D, optimizer, scheduler = get_models()

In [ ]:
#TODO wrap model so that it produces outputs from random inputs

Construct losses as defined above

In [ ]:
# TODO weights
loss_g = adversarial.AmbientGANGeneratorLoss(...)
loss_d = adversarial.AmbientGANDiscriminatorLoss(...)

Train the networks using `AdversarialTrainer`

In [ ]:
dinv.training.adversarial_training.AdversarialTrainer(
    D=D,
    epochs=5,
    losses=loss_g,
    losses_d=loss_d,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    verbose=True
).train(
    model=model,
    physics=physics,
    train_dataloader=train_dataloader,
    eval_dataloader=test_dataloader
)